# Step 2: Create and Push a Batch Run to AWS

**WARNING**: Make sure everything is set correctly BEFORE running this notebook!  Since this notebook starts processing on the AWS servers, it has the potential  to consume a lot of resources, aka dollars...

This notebook:
* Imports the ASF database query file we created in step 1
* chooses a set of interferograms to process
* prepares the files needed to process those interferograms
* uploads files to AWS servers and starts the processing

Requires the following files to be in the folder where you run the code:
* apmb.geojson (file with coordinates for APMB)
* query.geojson (file with results of ASF database query)
* template.yml (yaml file with ISCE processing parameters)

## 2.1: Import Python packages

Import the packages needed to run this notebook.  You may need to install the dinosar package first.

In [ ]:
# if dinosar library not in base environment uncomment below (run just once)
#!pip install --no-cache git+https://github.com/scottyhq/dinosar.git@master

In [14]:
import subprocess
import os
import dinosar
import geopandas as gpd
import getpass

ModuleNotFoundError: No module named 'dinosar'

## 2.2: Choose processing parameters

Make sure you have this section set the way you want it before running later cells!  

In this section we'll set:
* Which interferograms to make
* ISCE processing parameters (swaths, filters, etc.)
* AWS processing parameters (S3 bucket name, AWS job name)

### 2.2.1 Choose which interferograms to process
This section sets the track to process, and makes pairs from one date each month for all the data from that track.

Choose which track to process:

In [ ]:
# choose track to process
track = 83
# load the ASF search results that you generated in Step 1
gf = dinosar.archive.asf.load_inventory('query.geojson')
# create a new dataframe with only the track selected, in the date bounds selected
gdf=gf.query('relativeOrbit == @track')

This section is currently set up to then make sequential pairs with one date from each month.  However, that can be changed in future!

In [ ]:
# create yet another dataframe with only some info so we can quickly check that we selected the data we actually want
df = gdf.loc[:,['frameNumber','dateStamp','relativeOrbit']].sort_values(by='dateStamp')
df.head()

In [8]:
# Take only unique dates and set DateTimeIndex
df  = df.drop_duplicates('dateStamp').set_index('dateStamp', drop=False)
df.head()

,frameNumber,dateStamp,platform
dateStamp,,,
2014-11-03,1105,2014-11-03,Sentinel-1A
2014-11-27,1110,2014-11-27,Sentinel-1A
2014-12-21,1105,2014-12-21,Sentinel-1A
2015-01-14,1104,2015-01-14,Sentinel-1A
2015-02-07,1105,2015-02-07,Sentinel-1A


In [14]:
# create a function that will create pairs with one acquisition from each month
def one_per_month(df):
    # Single date per month
    tmp = df.resample('M').first()
    tmp1 = tmp.sort_index(ascending=False)
    dates = tmp1.dateStamp.apply(lambda x: x.strftime('%Y%m%d')).to_list()
    pairs = dates_to_pairs(dates)
    
    return pairs

In [ ]:
# create the pairs
pairs = one_per_month(df)
# print the number of pairs and a list of the pairs
print(f'{len(pairs)} Interferograms to generate:')
pairs

### 2.2.2: Set ISCE Processing Parameters

1. Open the "template.yml" file and save a copy with a new name for this batch run (e.g., "Template_83_JanFeb2019.yml").
2. In the new .yml file, change the swath numbers to match the track you want to process:
  * 156: 1,2
  * 149: 2,3
  * 76: 1,2
  * 83: 2,3
3. Optional - change other ISCE processing parameters in the template as needed.

In [5]:
# enter the name of the .yml template file here
template="Template_83_JanFeb2019.yml"

### 2.2.3: Set AWS parameters
These are the parameters you need to set for each job you push to AWS:
* Destination for processing files on AWS (folder name)
* Job name - to track your job while it is processing

Ideally, these names should tell you something about the processing job (i.e., "Track83_2014-2020_1month")

In [1]:
# aws parameters:
dirname = 'D83-JanFeb2019' #name of the folder on AWS where your processing files will be
jobname = 'uturuncu-D83-TEST' #name for the job on AWS

## 2.3: Create processing directories and push them to AWS

Now that we've defined our processing parameters and chosen the interferograms to process, we'll create a processing directory for each interferogram, and then upload those directories to the AWS server (S3).

In [ ]:
# set full path to bucket on S3
bucket = 's3://dinosar/processing/uturuncu/D83-JanFeb2019' + dirname
# create a text file with the interferogram pairs to process
pairsFile = 'pairs.txt'

paths = [bucket+'/'+x for x in pairs]
with open(pairsFile, 'w') as f:
    f.write('\n'.join(paths))

cmd = f'aws s3 cp {pairsFile} {bucket}/{pairsFile}'#write the command to push pairs.txt to the AWS bucket

print(cmd)

subprocess.call(cmd, shell=True) # runs the command to push the files with pairs to process to the AWS bucket

In [ ]:
with open(pairsFile) as f:
    pairs = [line.rstrip() for line in f]
    mapping = dict(enumerate(pairs))
mapping

In [ ]:
script = 'prep_topsApp_local'
for i,p in enumerate(pairs):
    intname = os.path.basename(p)
    junk,master,slave=intname.split('-')
    intdir = f'int-{master}-{slave}'
    cmd = f'{script} -i query.geojson -m {master} -p {relOrbit} -s {slave} -t {template}'
    print(i, cmd)
    subprocess.call(cmd, shell=True) # runs the command to make processing directories on the local system  

Now we should have a processing directory for each interferogram in this directory.  Each processing directory should have two files:
* topsApp.xml  = input file for ISCE processing
* download-links.txt = text file with the links to download all the data we'll need for processing

Now we push all those directories to the S3 cloud storage on AWS:

In [ ]:
# Move these to cloud storage
# Push folder of text files to S3
for i,p in enumerate(pairs):
    intname = os.path.basename(p)
    junk,master,slave=intname.split('-')
    intdir = f'int-{master}-{slave}'
    cmd = f'aws s3 sync {intdir}/ {bucket}/{intdir}/'
    print(cmd)
    subprocess.call(cmd, shell=True)

print(f'Moved files to {bucket}')

## 2.4: Launch Processing on AWS (WARNING: can consume lots of AWS resources!!)
Now that we have all the files we need for processing on the AWS servers, we can start processing!  Don't run these cells until you're *SURE* you've got the interferograms you want!

In [ ]:
# Enter your NASA URS password to download SLCs
nasauser = 'pmacqueen' # NASA EarthData username
nasapass = getpass.getpass() # NASA EarthData password (will create an interactive textbox)

In [ ]:
# don't change these:
demDir = 's3://dinosar/processing/uturuncu/dem' #where the DEM is stored on AWS
jobdef = 'uturuncu-array' # sets certain parameters on AWS
jobqueue = 'uturuncu-queue' # sets certain parameters on AWS
array_size = len(pairs)


# NOTE: job-name, job-queue, and job-definition are JSON files that I've created for AWS Batch
# The specify type of computers to use, etc
cmd = f"aws batch submit-job \
--job-name {jobname} \
--job-queue {jobqueue} \
--job-definition {jobdef} \
--array-properties size={array_size} \
--parameters 'S3_PAIRS={bucket}/{pairsFile},S3_DEM={demDir}' \
--container-overrides 'environment=[{{name=NASAUSER,value={nasauser}}},{{name=NASAPASS,value={nasapass}}}]' \
"

# WARNING: this prints your password as plain text, careful not to push to github
# If you run the command in terminal sometimes the error messages are more helpful
#print(cmd) # uncomment to print the command for debugging purposes.

In [30]:
# Run this cell to start processing!
aws_batchout = subprocess.check_output(cmd, shell=True) # runs the AWS command to start the batch job
aws_batchout # prints out the job-id - make a note of the job id for finding the logs later!


In [11]:
# Write a text file with a summary of this job for reference later
sumfilename = 'summary_' + jobname + '.txt'

with open(sumfilename, 'w') as sf:
    sf.write('track = '+ str(track) + '\n')
    sf.write('template file = '+ template + '\n')
    sf.write('S3 Bucket Name = ' + dirname + '\n')
    sf.write('Job Name = ' + jobname)
    sf.write('Job Name and Job ID on AWS = ' + aws_batchout)

## 2.5: Wait for the jobs to finish!

* The end products will be in "s3://dinosar/results/uturuncu/(dirname)/(int_dir)/merged"
* Check the file "summary_(jobname).txt" for a summary of the processing parameters you set in this script
* You can monitor jobs here: https://us-west-2.console.aws.amazon.com/batch/home?region=us-west-2#/jobs/queue/arn:aws:batch:us-west-2:783380859522:job-queue~2Futuruncu-queue?state=PENDING
* After 24 hours, you'll have to look up your job here using the job id: https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logStream:group=/aws/batch/job;streamFilter=typeLogStreamPrefix